In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import (VGG16, VGG19, ResNet50, InceptionV3, MobileNetV2,
                                DenseNet121, Xception)
from keras.applications.efficientnet import (EfficientNetB0, EfficientNetB1, EfficientNetB2,
                                             EfficientNetB3, EfficientNetB4, EfficientNetB5,
                                             EfficientNetB6, EfficientNetB7, preprocess_input as efficientnet_preprocess_input)
from keras.applications.vgg16 import preprocess_input as vgg16_preprocess_input
from keras.applications.vgg19 import preprocess_input as vgg19_preprocess_input
from keras.applications.resnet50 import preprocess_input as resnet50_preprocess_input
from keras.applications.inception_v3 import preprocess_input as inceptionv3_preprocess_input
from keras.applications.mobilenet_v2 import preprocess_input as mobilenetv2_preprocess_input
from keras.applications.densenet import preprocess_input as densenet_preprocess_input
from keras.applications.xception import preprocess_input as xception_preprocess_input
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.layers import BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense
import matplotlib.pyplot as plt
import seaborn as sns
from keras.optimizers import Adam
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, precision_score

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Constants
DATA_DIR = "/kaggle/input/ct-kidney-dataset-normal-cyst-tumor-and-stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"


In [4]:
# Model configurations
num_classes = 4
batch_size = 32
epochs = 50

In [5]:
# Map model names to their function, preprocess_input, and input_shape
model_info = {
    'VGG16': (VGG16, vgg16_preprocess_input, (224, 224, 3)),
    'VGG19': (VGG19, vgg19_preprocess_input, (224, 224, 3)),
    'ResNet50': (ResNet50, resnet50_preprocess_input, (224, 224, 3)),
    'InceptionV3': (InceptionV3, inceptionv3_preprocess_input, (299, 299, 3)),
    'MobileNetV2': (MobileNetV2, mobilenetv2_preprocess_input, (224, 224, 3)),
    'DenseNet121': (DenseNet121, densenet_preprocess_input, (224, 224, 3)),
    'Xception': (Xception, xception_preprocess_input, (299, 299, 3)),
    'EfficientNetB0': (EfficientNetB0, efficientnet_preprocess_input, (224, 224, 3)),
    'EfficientNetB1': (EfficientNetB1, efficientnet_preprocess_input, (240, 240, 3)),
    'EfficientNetB2': (EfficientNetB2, efficientnet_preprocess_input, (260, 260, 3)),
    'EfficientNetB3': (EfficientNetB3, efficientnet_preprocess_input, (300, 300, 3)),
    'EfficientNetB4': (EfficientNetB4, efficientnet_preprocess_input, (224, 224, 3))),
    'EfficientNetB5': (EfficientNetB5, efficientnet_preprocess_input, (224, 224, 3))),
    'EfficientNetB6': (EfficientNetB6, efficientnet_preprocess_input, (224, 224, 3))),
    'EfficientNetB7': (EfficientNetB7, efficientnet_preprocess_input, (224, 224, 3)))
}

In [6]:
def train_model(model_name, num_classes, data_dir, epochs, batch_size):
    model_class, preprocess_input, input_shape = model_info[model_name]

    # Load and preprocess the data using tf.data
    train_dataset = image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="training",
        seed=123,
        image_size=input_shape[:2],
        batch_size=batch_size,
        label_mode='categorical'
    )

    validation_dataset = image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="validation",
        seed=123,
        image_size=input_shape[:2],
        batch_size=batch_size,
        label_mode='categorical'
    )

    # Define the image augmentation layer
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
        tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
        tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1)
    ])

    # Apply augmentation and preprocessing only to the training dataset
    train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y))
    train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y))

    # Apply only preprocessing to the validation dataset
    validation_dataset = validation_dataset.map(lambda x, y: (preprocess_input(x), y))

    # AUTOTUNE
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
    validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)


    # Model creation
    base_model = model_class(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Compile the model
    METRICS = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=METRICS)
    
    
    # Define Callbacks
    checkpoint = ModelCheckpoint(filepath=f'{model_name}_model.h5',
                                 monitor='val_accuracy',
                                 mode='max',
                                 save_best_only=True,
                                 verbose=1)

    earlystop = EarlyStopping(monitor='val_accuracy',
                              min_delta=0.001,
                              patience=15,
                              restore_best_weights=True)

    reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.1,
                                  patience=10,
                                  verbose=1,
                                  min_delta=0.0001,
                                  min_lr=0.0001)

    callbacks = [checkpoint, earlystop, reduce_lr]
    
    # Train the model
    history = model.fit(
        train_dataset,
        epochs=epochs,
        validation_data=validation_dataset,
        callbacks=callbacks
    )
    
    # Plot training history
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{model_name} Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{model_name} Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Save the plot
    plt.savefig(f'{model_name}_accuracy_loss_plot.png')
    plt.close()
    
    fig, ax = plt.subplots(2,2 , figsize=(8, 10))
    ax = ax.ravel()

    for i, met in enumerate(['precision', 'recall', 'accuracy', 'loss']):
        ax[i].plot(history.history[met])
        ax[i].plot(history.history['val_' + met])
        ax[i].set_title('Model {}'.format(met))
        ax[i].set_xlabel('epochs')
        ax[i].set_ylabel(met)
        ax[i].legend(['train', 'val'])
    fig.savefig(f'{model_name}_metrics.png')
    
    print(f"{model_name}_performance...")
    loss, accuracy, precision, recall = model.evaluate(validation_dataset)
    print(f"Validation Loss: {loss}")
    print(f"Validation Accuracy: {accuracy}")
    print(f"Validation Precision: {precision}")
    print(f"Validation Recall: {recall}")
    
    val_dataset = validation_dataset.unbatch().batch(1)
    true_labels = []
    predictions = []
    for x, y in val_dataset:
        true_labels.append(np.argmax(y.numpy(), axis=1))
        predictions.append(np.argmax(model.predict(x), axis=1))

    true_labels = np.concatenate(true_labels)
    predictions = np.concatenate(predictions)

    # Confusion Matrix
    cm = confusion_matrix(true_labels, predictions)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Cyst', 'Normal', 'Stone', 'Tumor'],yticklabels=['Cyst', 'Normal', 'Stone', 'Tumor'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix for {model_name}')
    plt.savefig(f'{model_name}_confusion_matrix.png')

    # Metrics
    print(f"{model_name}_performance Metrics...")
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='weighted')
    recall = recall_score(true_labels, predictions, average='weighted')
    f1score = f1_score(true_labels, predictions, average='weighted')
    
    print("accuracy", accuracy)
    print("precision", precision)
    print("recall", recall)
    print("f1-score", f1score)

    # Save metrics to CSV file
    csv_file = 'model_results.csv'
    new_row = pd.DataFrame([[model_name, accuracy, precision, recall, f1score]], columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])
    if not os.path.exists(csv_file):
        new_row.to_csv(csv_file, index=False)
    else:
        pd.concat([pd.read_csv(csv_file), new_row]).to_csv(csv_file, index=False)
    
    return model

In [ ]:
# Train and save each model
for model_name in model_info.keys():
    print(f"Training {model_name}...")
    model = train_model(model_name, num_classes, DATA_DIR, epochs, batch_size)
#     model.save(f'{model_name}_model.h5')
    print(f"{model_name} model training completed and saved.")
    print("=============================================================================")

Training VGG16...
Found 12446 files belonging to 4 classes.
Using 9957 files for training.
Found 12446 files belonging to 4 classes.
Using 2489 files for validation.
58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/50
312/312 [==============================] - ETA: 0s - loss: 1.2137 - accuracy: 0.5755 - precision: 0.6390 - recall: 0.4620
Epoch 1: val_accuracy improved from -inf to 0.41382, saving model to VGG16_model.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


312/312 [==============================] - 251s 732ms/step - loss: 1.2137 - accuracy: 0.5755 - precision: 0.6390 - recall: 0.4620 - val_loss: 1.8086 - val_accuracy: 0.4138 - val_precision: 0.5082 - val_recall: 0.3973 - lr: 0.0010
Epoch 2/50
312/312 [==============================] - ETA: 0s - loss: 0.8985 - accuracy: 0.6682 - precision: 0.7340 - recall: 0.5621
Epoch 2: val_accuracy improved from 0.41382 to 0.45440, saving model to VGG16_model.h5
312/312 [==============================] - 141s 453ms/step - loss: 0.8985 - accuracy: 0.6682 - precision: 0.7340 - recall: 0.5621 - val_loss: 1.6454 - val_accuracy: 0.4544 - val_precision: 0.5756 - val_recall: 0.3961 - lr: 0.0010
Epoch 3/50
312/312 [==============================] - ETA: 0s - loss: 0.7542 - accuracy: 0.6866 - precision: 0.7528 - recall: 0.6053
Epoch 6: val_accuracy did not improve from 0.67658
312/312 [==============================] - 140s 448ms/step - loss: 0.7542 - accuracy: 0.6866 - precision: 0.7528 - recall: 0.6053 - val_